# Testing GMM models on the test set

### Ensure 'train.ipynb' is run before running this file

In [1]:
# Required libraries to import
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import pickle
import per # Library to perform phoneme error recognition

In [2]:
# Accordingly change the mfcc_dir, TRAIN_DF, TEST_DF for the number of components and number of mfcc features

mfcc_dir = 'models/mfcc_13_2/'
TRAIN_DF = 'features/train1.hdf'
TEST_DF = 'features/test1.hdf'
ENERGY_FLAG = True # To include energy co-efficient or not
TRAIN_FLAG = False # To perform training or not

In [3]:
# Importing the hdf file got by generation from the import_timit.py file 
timit_test_df = pd.read_hdf(TEST_DF)

In [4]:
# Show the first 5 rows of the table
timit_test_df.head()

,features,labels
0,"[8.432898672227827, -17.192751467628792, -26.4...",sil
1,"[8.174287614589556, -13.339226994007353, -24.3...",sil
2,"[8.081531888228382, -13.91009808149714, -18.14...",sil
3,"[8.01702587918423, -17.681371055083115, -21.71...",sil
4,"[7.951984723992553, -20.67793996977594, -26.00...",sil


In [5]:
# Extracting features and labels from the test set
test_features = np.array(timit_test_df["features"].tolist())
test_labels = np.array(timit_test_df["labels"].tolist())

In [6]:
np.shape(test_features)

(451660, 13)

In [7]:
# Deleting energy co-efficients from test file

if not ENERGY_FLAG:
    
    rem = [0] # Change rem to [0], [0,12] or [0,12,24] for 13, 24 and 39 mfcc features respectively

    for i in rem:
        test_features = np.delete(test_features,i,1)

In [8]:
np.shape(test_features)

(451660, 13)

In [9]:
# Replacing the blank phoneme label with the word 'blank'
sortlabels = sorted(list(set(test_labels)))
sortlabels[0] = 'blank'
sortlabels

['blank',
 'aa',
 'ae',
 'ah',
 'aw',
 'ay',
 'b',
 'ch',
 'd',
 'dh',
 'dx',
 'eh',
 'er',
 'ey',
 'f',
 'g',
 'hh',
 'ih',
 'iy',
 'jh',
 'k',
 'l',
 'm',
 'n',
 'ng',
 'ow',
 'oy',
 'p',
 'r',
 's',
 'sh',
 'sil',
 't',
 'th',
 'uh',
 'uw',
 'v',
 'w',
 'y',
 'z']

In [10]:
# Creating array to store all models
gmm_models=[]

# Loading and storing all 40 models into array
for i in sortlabels:
    with open(mfcc_dir+i+'.pkl','rb') as pkl_file:
        gmm_models.append(pickle.load(pkl_file))

### Calculating Frame-level / Testing Accuracy

In [11]:
# Frame-level Accuracy (Testing Accuracy)
test_scores=[]
# MAP calculation for each of the 40 models. Finally an array of arrays is got
for i in range(len(gmm_models)):
    print("MAP calculation for GMM model of phoneme ",i+1,': ',sortlabels[i])
    test_scores.append(gmm_models[i].score_samples(test_features))

MAP calculation for GMM model of phoneme  1 :  blank
MAP calculation for GMM model of phoneme  2 :  aa
MAP calculation for GMM model of phoneme  3 :  ae
MAP calculation for GMM model of phoneme  4 :  ah
MAP calculation for GMM model of phoneme  5 :  aw
MAP calculation for GMM model of phoneme  6 :  ay
MAP calculation for GMM model of phoneme  7 :  b
MAP calculation for GMM model of phoneme  8 :  ch
MAP calculation for GMM model of phoneme  9 :  d
MAP calculation for GMM model of phoneme  10 :  dh
MAP calculation for GMM model of phoneme  11 :  dx
MAP calculation for GMM model of phoneme  12 :  eh
MAP calculation for GMM model of phoneme  13 :  er
MAP calculation for GMM model of phoneme  14 :  ey
MAP calculation for GMM model of phoneme  15 :  f
MAP calculation for GMM model of phoneme  16 :  g
MAP calculation for GMM model of phoneme  17 :  hh
MAP calculation for GMM model of phoneme  18 :  ih
MAP calculation for GMM model of phoneme  19 :  iy
MAP calculation for GMM model of phoneme 

In [12]:
# TESTING ACCURACY
# Predicting labels using argmax indices
pred_test_labels=np.array(sorted(list(set(test_labels))))[np.argmax((np.transpose(test_scores)),axis=1)]
# Accuracy
test_acc = (np.count_nonzero(pred_test_labels==test_labels)/len(test_labels))*100
print("Frame-level Accuracy: ",test_acc,"%")

Frame-level Accuracy:  11.619359695346057 %


In [18]:
# Phoneme error rate

phoneme_error_rate = per.main(test_labels, pred_test_labels)

print("Phoneme Error Rate: ",phoneme_error_rate*100)

Phoneme Error Rate:  90.55478929984393


### END OF TESTING